## Delhi high court

In [ ]:
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize WebDriver
driver = webdriver.Chrome()

# Open the main page
driver.get("https://dhcbkp.nic.in/FreeText/launchbrowsejud.do")  # Change this to your actual webpage

# Wait for all <span> elements containing <a> tags
wait = WebDriverWait(driver, 10)
span_elements = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "span")))

all_links = []  # List to store extracted hrefs

# Iterate over all spans and find <a> tags with onclick
for span in span_elements:
    try:
        a_tag = span.find_element(By.TAG_NAME, "a")  # Find <a> inside <span>
        onclick_text = a_tag.get_attribute("onclick")  # Extract onclick attribute

        # Extract the URL inside setiframe('URL')
        match = re.search(r"setiframe\('(.+?)'\)", onclick_text)
        if match:
            print(f"Clicking on link with URL: {match.group(1)}")
            a_tag.click()  # Click the link to navigate
            
            # Wait for the first table to load
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "table")))

            # Find the first <table> element
            table = driver.find_element(By.TAG_NAME, "table")
            print(table.text)  # Print the table text


            links = table.find_elements(By.TAG_NAME, "a")
            
            for link in links:
                href = link.get_attribute("href")
                if href:
                    all_links.append(href)

    except Exception as e:
        print("Error processing link:", e)

# Close browser
driver.quit()

# Save all links to a CSV file
csv_filename = "extracted_links.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Links"])  # Header row
    for link in all_links:
        writer.writerow([link])

print(f"✅ All links saved to {csv_filename}")

In [ ]:
import os
import requests
import pandas as pd

def download_pdf(url, folder):
    # Get the file name from the URL
    file_name = url.split('/')[-1]
    file_path = os.path.join(folder, file_name)

    try:
        # Send HTTP request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)

        # Write the content of the response to a file
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f'Successfully downloaded: {file_name}')
    except requests.exceptions.RequestException as e:
        print(f'Failed to download: {file_name}. Error: {e}')

# Define the URL of the PDF and the folder to save it
#url = 'https://main.sci.gov.in/judgment/judis/'
folder = 'delhi1'
df=pd.read_csv("pdf_links.csv")
# Create the folder if it doesn't exist
if not os.path.exists(folder):
    os.makedirs(folder)


li=df['PDF Link']
print(len(li))
# Download the PDF
count=0
for i in li:
    print(count)
    #ur=url+str(i)+".pdf"
    if('judgement' in i):
        download_pdf(i, folder)
    count+=1



## Moving files between folders

In [ ]:
import os
import shutil

source_folder = 'madras1'
destination_folder = 'madras'

# Ensure the destination folder exists
os.makedirs(destination_folder, exist_ok=True)

# Iterate over all files in the source folder
for filename in os.listdir(source_folder):
    source_file = os.path.join(source_folder, filename)
    destination_file = os.path.join(destination_folder, filename)
    
    # Move file
    shutil.move(source_file, destination_file)

print(f"All files have been moved from {source_folder} to {destination_folder}")


## No of files in a folder

In [ ]:
import os


folder_path = 'madras'
files = os.listdir(folder_path)
num_files = len(files)
print(f"Number of files in folder: {num_files}")

In [ ]:
import os
folder_path = 'madras1'  
files = os.listdir(folder_path)
num_files = len(files)
files.sort(reverse=True)
print(files[0])
print(f"Number of files in folder: {num_files}")
last_file = files[-1] if num_files > 0 else None
print(f"Last file name: {last_file}")


In [ ]:
import os

# Function to get the size of a file
def get_file_size(file_path):
    try:
        # Get the size of the file in bytes
        size_bytes = os.path.getsize(file_path)
        # Convert bytes to a more readable format (optional)
        size_kb = size_bytes / 1024 
        return size_kb
    except Exception as e:
        print(f"Error getting file size: {str(e)}")
        return None
li=[]
for i in files:
    size=get_file_size('madras/'+(i))
    if(size<1):
        li.append(i)


In [ ]:
print(len(li))

In [ ]:
import os
def find_duplicates(lst):
    seen = set()
    duplicates = set()
    for item in lst:
        if item in seen:
            duplicates.add(item)
        else:
            seen.add(item)
    return list(duplicates)

# Example usage
folder_path = 'LatestLaws' 
my_list = os.listdir(folder_path)
duplicates = find_duplicates(my_list)
print(duplicates)  # Output: [1, 2, 3]


In [ ]:
folder_path = 'LatestLaws' 
my_list = os.listdir(folder_path)
print(len(my_list))
def find_pdfs(lst):
    return [item for item in lst if item.endswith(').pdf')]

print(len(find_pdfs(my_list)))

## delete files

In [ ]:
import os
import glob

def delete_specific_files(folder_path, pattern):
    files = glob.glob(os.path.join(folder_path, pattern))
    for file in files:
        try:
            os.remove(file)
            print(f"Deleted: {file}")
        except Exception as e:
            print(f"Failed to delete {file}. Reason: {e}")

# Example usage
folder_path = 'Patna'
pattern = '*).PDF'  # Specify your pattern here
delete_specific_files(folder_path, pattern)


## ZIP extraction

In [ ]:
import zipfile
import os

zip_file_path = 'patna2018.zip'
extract_to_path = 'Patna'

# Ensure the extraction folder exists
os.makedirs(extract_to_path, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f"Files have been extracted to {extract_to_path}")


## AP high court

In [ ]:
import os
import requests
from bs4 import BeautifulSoup

def download_pdf(url, folder):
    # Get the file name from the URL
    file_name = url.split('/')[-1]
    file_path = os.path.join(folder, file_name)
    
    try:
        # Send HTTP request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
        
        # Write the content of the response to a file
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f'Successfully downloaded: {file_name}')
    except requests.exceptions.RequestException as e:
        print(f'Failed to download: {file_name}. Error: {e}')

def scrape_and_download(base_url, folder):
    try:
        # Send HTTP request to the base URL
        response = requests.get(base_url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
        
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all links on the page
        links = soup.find_all('a', href=True)
        
        # Filter links that end with .pdf
        pdf_links = [link['href'] for link in links if link['href'].endswith('.pdf')]
        
        # Download each PDF found
        for link in pdf_links:
            # Ensure the link is an absolute URL
            if not link.startswith('http'):
                link = base_url + link
            download_pdf(link, folder)
    except requests.exceptions.RequestException as e:
        print(f'Failed to access the base URL: {base_url}. Error: {e}')

# Define the base URL and the folder to save the PDFs
base_url = 'https://aphc2.in/hcap_judgments/docs/sci/eng/'
folder = 'ap'

# Create the folder if it doesn't exist
if not os.path.exists(folder):
    os.makedirs(folder)

# Call the function to scrape and download PDFs
scrape_and_download(base_url, folder)


## Telangana High Court

In [ ]:
import os
import requests
import pandas as pd

def download_pdf(url, folder):
    # Get the file name from the URL
    file_name = url.split('/')[-1]
    file_path = os.path.join(folder, file_name)

    try:
        # Send HTTP request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)

        # Write the content of the response to a file
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f'Successfully downloaded: {file_name}')
    except requests.exceptions.RequestException as e:
        print(f'Failed to download: {file_name}. Error: {e}')

# Define the URL of the PDF and the folder to save it
#url = 'https://main.sci.gov.in/judgment/judis/'
folder = 'ts'
df=pd.read_csv("tsjuglinks.csv")
# Create the folder if it doesn't exist
if not os.path.exists(folder):
    os.makedirs(folder)


li=df['tslinks']
print(len(li))
# Download the PDF
count=0
for i in li:
    print(count)
    #ur=url+str(i)+".pdf"
    if('telugu' not in i):
        download_pdf(i, folder)
    count+=1

print(count)

## Gauhati high court

In [ ]:
import os
import requests
import pandas as pd

def download_pdf(url, folder):
    # Get the file name from the URL
    file_name = url.split('/')[-1]
    file_path = os.path.join(folder, file_name)

    try:
        
        # Send HTTP request to the URL
        response = requests.get(url)
        print('hai')
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
        print(response)
        # Write the content of the response to a file
        with open(file_path, 'wb') as f:
            print("hi")
            f.write(response.content)
        print(f'Successfully downloaded: {file_name}')
    except requests.exceptions.RequestException as e:
        print(f'Failed to download: {file_name}. Error: {e}')

# Define the URL of the PDF and the folder to save it
#url = 'https://main.sci.gov.in/judgment/judis/'
folder = 'gauhati'
df=pd.read_csv("extracted_urls.csv")
# Create the folder if it doesn't exist
if not os.path.exists(folder):
    os.makedirs(folder)


li=df['URLs']
print(len(li))
# Download the PDF
count=0
for i in li:
    print(count)
    #ur=url+str(i)+".pdf"
    
    download_pdf(i, folder)
    count+=1

print(count)

In [ ]:
import fitz  # PyMuPDF
import pandas as pd

def extract_urls_from_pdf(pdf_path):
    # Open the PDF file
    document = fitz.open(pdf_path)
    
    urls = []
    
    # Iterate through each page
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        links = page.get_links()
        
        for link in links:
            uri = link.get("uri", None)
            if uri:
                urls.append(uri)
    
    return urls

# Path to the PDF file
pdf_path = "JDMTJan17.pdf"

# Extract URLs from the PDF
urls = extract_urls_from_pdf(pdf_path)

# Create a dataframe and save to CSV
print(urls)
df = pd.DataFrame(urls, columns=["URLs"])
df.to_csv("extracted_urls.csv", index=False)

print(f"Extracted {len(urls)} URLs from the PDF.")


In [ ]:
download_pdf('https://ghconline.gov.in/JDMTFeb17.pdf','static')

## madras high court

In [ ]:
import os
import requests
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")
# Function to download and save PDF
def download_pdf(pdf_url, output_folder):
    try:
        # Get the PDF filename from the URL
        file_name = pdf_url.split("/")[-1]+'.pdf'
        # Construct the full file path
        full_file_path = os.path.join(output_folder, file_name)
        
        # Send a GET request to the PDF URL
        response = requests.get(pdf_url, stream=True, verify=False)  # Disable SSL verification
        
        # Check if the request was successful
        if response.status_code == 200:
            # Write the PDF content to a file
            with open(full_file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        f.write(chunk)
            #print(f"Downloaded PDF from {pdf_url} to {full_file_path}")
        else:
            print(f"Failed to download PDF from {pdf_url} (Status Code: {response.status_code})")
    except Exception as e:
        print(f"Error downloading PDF from {pdf_url}: {str(e)}")

# List of PDF URLs to download

# Folder to save downloaded PDFs
output_folder = 'madras1'
os.makedirs(output_folder, exist_ok=True)

# Download each PDF from the list of URLs
for i in range(320567,325000):
    pdf_url="https://mhc.tn.gov.in/judis/index.php/casestatus/viewpdf/"+str(i)
    download_pdf(pdf_url, output_folder)


In [ ]:
print(i)

In [ ]:
import os
folder_path = 'madras'  
files = os.listdir(folder_path)
num_files = len(files)
files.sort(reverse=True)
print(files[0])
print(f"Number of files in folder: {num_files}")
last_file = files[-1] if num_files > 0 else None
print(f"Last file name: {last_file}")


In [ ]:
import pdfplumber
import pandas as pd
import os

df=pd.DataFrame()
def extract_text_between_headings(pdf_path):
    try:
        pdf = pdfplumber.open(pdf_path)
    except FileNotFoundError:
        print(f"The file {pdf_path} was not found.")
        return ""
    except Exception as e:
        print(f"An error occurred while opening the file: {e}")
        return ""

    text = ""
    for page in pdf.pages:
        page_text =page.extract_text()
        lines = page_text.split('\n')
        for line in lines:
            #print(line)
           
            if line=="ORDER" or line== "O R D E R" or line=="COMMON ORDER" or line=="C O M M O N O R D E R":
               print(line)
               return None
    return text.strip()

l=[]
for i in files[90000:120000]:
  pdf_path = 'madras/'+i
  extracted_text = extract_text_between_headings(pdf_path)
  if((extracted_text)!=None):
    l.append(i)



In [ ]:
import shutil
import os

# Define source and destination folders
source_folder = 'madras'
destination_folder = 'madras_judgments'

# Ensure the destination folder exists
os.makedirs(destination_folder, exist_ok=True)

# Move files
for filename in l:
    source_file = os.path.join(source_folder, filename)
    destination_file = os.path.join(destination_folder, filename)
    shutil.move(source_file, destination_file)

print("Files moved successfully!")
